## 1.1 - Movielens full - EDA

---

In [1]:
import pandas as pd
import seaborn as sns
import plotly.express as px

In [2]:
movies_path = "../Data/ml-latest/movies.csv"
ratings_path = "../Data/ml-latest/ratings.csv"

df_movies = pd.read_csv(movies_path, usecols=["movieId", "title"], dtype= {"movieId": "int32", "title": "str"})
df_ratings = pd.read_csv(ratings_path, usecols=["userId", "movieId", "rating"], dtype= {"userId": "int32", "movieId": "int32", "rating": "float32"})

### **a)** 

### Gör en EDA för att förstå datasetet. Inkludera olika slags plots. Begränsa dig inte till frågorna nedan, utan försök undersöka fler aspekter av datan.

---

In [3]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int32 
 1   title    58098 non-null  object
dtypes: int32(1), object(1)
memory usage: 681.0+ KB


In [4]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int32  
 1   movieId  int32  
 2   rating   float32
dtypes: float32(1), int32(2)
memory usage: 317.6 MB


In [5]:
df_movies.describe()

,movieId
count,58098.000000
mean,111919.516197
std,59862.660956
min,1.000000
25%,72437.750000
50%,126549.000000
75%,161449.500000
max,193886.000000


In [6]:
df_ratings.describe()

,userId,movieId,rating
count,2.775344e+07,2.775344e+07,2.775344e+07
mean,1.419420e+05,1.848800e+04,3.530446e+00
std,8.170740e+04,3.510263e+04,1.066353e+00
min,1.000000e+00,1.000000e+00,5.000000e-01
25%,7.117600e+04,1.097000e+03,3.000000e+00
50%,1.420220e+05,2.716000e+03,3.500000e+00
75%,2.124590e+05,7.150000e+03,4.000000e+00
max,2.832280e+05,1.938860e+05,5.000000e+00


### **b)** 
### Vilka är de 10 filmerna med flest ratings?

---

In [7]:
df_num_ratings = df_ratings.groupby(["movieId"], as_index=True).size()

df_top10_num_ratings = (
    df_num_ratings.sort_values(axis=0, ascending=False).head(10).reset_index()
)

for i in range(len(df_top10_num_ratings)):
    print(
        f"{i+1}: {(df_movies['title'].loc[df_movies['movieId'] == int(df_top10_num_ratings['movieId'].iloc[[i]])]).to_string(header=False, index=False)}, Ratings: {(df_top10_num_ratings[0].iloc[[i]]).to_string(header=False, index=False)}"
    )


1: Shawshank Redemption, The (1994), Ratings: 97999
2: Forrest Gump (1994), Ratings: 97040
3: Pulp Fiction (1994), Ratings: 92406
4: Silence of the Lambs, The (1991), Ratings: 87899
5: Matrix, The (1999), Ratings: 84545
6: Star Wars: Episode IV - A New Hope (1977), Ratings: 81815
7: Jurassic Park (1993), Ratings: 76451
8: Schindler's List (1993), Ratings: 71516
9: Braveheart (1995), Ratings: 68803
10: Toy Story (1995), Ratings: 68469


### **c)** 
### Beräkna den genomsnittliga ratingen för dessa 10 filmerna med flest ratings.

---

In [8]:
df_top10_mean_ratings = df_ratings.groupby(["movieId"], as_index=False).mean()

for i in range(len(df_top10_num_ratings)):
    print(
        f"{i+1}: {(df_movies['title'].loc[df_movies['movieId'] == int(df_top10_num_ratings['movieId'].iloc[[i]])]).to_string(header=False, index=False)}, Rating: {float((df_top10_mean_ratings['rating'].loc[df_top10_mean_ratings['movieId'] == int(df_top10_num_ratings['movieId'].iloc[[i]])]).to_string(header=False, index=False)):.1f}"
    )

1: Shawshank Redemption, The (1994), Rating: 4.4
2: Forrest Gump (1994), Rating: 4.1
3: Pulp Fiction (1994), Rating: 4.2
4: Silence of the Lambs, The (1991), Rating: 4.2
5: Matrix, The (1999), Rating: 4.1
6: Star Wars: Episode IV - A New Hope (1977), Rating: 4.1
7: Jurassic Park (1993), Rating: 3.7
8: Schindler's List (1993), Rating: 4.3
9: Braveheart (1995), Rating: 4.0
10: Toy Story (1995), Rating: 3.9


### **d)**
### Gör en plot över årtal och antalet filmer representerade i datasetet.

---

In [9]:
df_movies_year = df_movies
df_movies_year["year"] = df_movies["title"].str.extract(r'\((\d{4})\)')
df_movies_per_year = df_movies_year.groupby("year").size().reset_index(name="count")

fig = px.bar(df_movies_per_year, x="year", y= "count", title='Number of Movies per Year')
fig.show()

### **e)** 
### Gör en plot över antalet ratings mot movieId.
---

In [10]:
df_num_ratings = df_num_ratings.reset_index(name= "count")

In [13]:
fig = px.histogram(df_num_ratings, x="movieId", y="count", title='Number of Ratings per Movie')
fig.show()

### **f)** 
### Beräkna genomsnittliga ratings för de top 10 filmerna med flest ratings. Gör ett stapeldiagram över dessa.

---

In [30]:
df_mean_count_ratings = pd.merge(df_num_ratings, df_top10_mean_ratings, on="movieId", how="outer")
df_mean_count_ratings = pd.merge(df_mean_count_ratings, df_movies, on="movieId", how="outer")
df_mean_count_ratings = df_mean_count_ratings.sort_values(by="count", axis=0, ascending=False).head(10).reset_index()

In [32]:
fig = px.bar(df_mean_count_ratings, x="title", y="rating", title="Mean Rating of top 10 most rated Movies")
fig.show()